a) Import bibliotek

In [3]:
import pandas as pd
import numpy as np


b) Import danych

In [4]:
df = pd.read_csv('pacjenci_demo_system_ekspertowy.csv')
df.head()


,patient_id,age,bmi,glucose,systolic_bp,diastolic_bp
0,P01,24,21.7,168,176,83
1,P02,73,27.8,101,148,98
2,P03,65,21.2,96,102,63
3,P04,49,30.4,145,167,91
4,P05,49,25.8,140,149,87


c) Definicja bazy fakt?w

In [5]:
def _find_col(cols, keys):
    cols_l = {c.lower(): c for c in cols}
    for k in keys:
        for c in cols:
            if k in c.lower():
                return c
    return None

def _col_map(df):
    cols = list(df.columns)
    return {
        'age': _find_col(cols, ['age', 'wiek']),
        'bmi': _find_col(cols, ['bmi']),
        'sbp': _find_col(cols, ['sbp', 'systolic', 'skurcz']),
        'dbp': _find_col(cols, ['dbp', 'diastolic', 'rozkurcz']),
        'glucose': _find_col(cols, ['glucose', 'gluko', 'cukier'])
    }

col_map = _col_map(df)
missing = [k for k,v in col_map.items() if v is None]
if missing:
    raise ValueError(f'Missing columns for: {missing}')

def facts_from_row(row, col_map):
    return {
        'age': float(row[col_map['age']]),
        'bmi': float(row[col_map['bmi']]),
        'sbp': float(row[col_map['sbp']]),
        'dbp': float(row[col_map['dbp']]),
        'glucose': float(row[col_map['glucose']])
    }

facts = facts_from_row(df.iloc[0], col_map)
facts


{'age': 24.0, 'bmi': 21.7, 'sbp': 176.0, 'dbp': 83.0, 'glucose': 168.0}

d) Definicja klasycznych regu? ekspertowych (IF?THEN)

In [6]:
rules = [
    {
        'name': 'nadcisnienie',
        'if': lambda f: (f['sbp'] >= 140) or (f['dbp'] >= 90),
        'then': 'nadcisnienie'
    },
    {
        'name': 'otylosc',
        'if': lambda f: f['bmi'] >= 30,
        'then': 'otylosc'
    },
    {
        'name': 'cukrzyca',
        'if': lambda f: f['glucose'] >= 126,
        'then': 'cukrzyca'
    }
]
rules


[{'name': 'nadcisnienie',
  'if': <function __main__.<lambda>(f)>,
  'then': 'nadcisnienie'},
 {'name': 'otylosc', 'if': <function __main__.<lambda>(f)>, 'then': 'otylosc'},
 {'name': 'cukrzyca',
  'if': <function __main__.<lambda>(f)>,
  'then': 'cukrzyca'}]

e) Mechanizm wnioskowania w prz?d (forward chaining)

In [7]:
def forward_chaining(facts, rules):
    conclusions = set()
    changed = True
    while changed:
        changed = False
        for r in rules:
            if r['if'](facts) and r['then'] not in conclusions:
                conclusions.add(r['then'])
                changed = True
    return list(conclusions)

conclusions = forward_chaining(facts, rules)
conclusions


['cukrzyca', 'nadcisnienie']

f) Regu?y z wagami (pewno?? regu?)

In [8]:
weighted_rules = [
    {
        'name': 'nadcisnienie',
        'if': lambda f: (f['sbp'] >= 140) or (f['dbp'] >= 90),
        'then': 'nadcisnienie',
        'weight': 0.9
    },
    {
        'name': 'otylosc',
        'if': lambda f: f['bmi'] >= 30,
        'then': 'otylosc',
        'weight': 0.8
    },
    {
        'name': 'cukrzyca',
        'if': lambda f: f['glucose'] >= 126,
        'then': 'cukrzyca',
        'weight': 0.85
    }
]

def weighted_inference(facts, rules):
    conclusions = {}
    for r in rules:
        if r['if'](facts):
            conclusions[r['then']] = max(conclusions.get(r['then'], 0.0), r['weight'])
    return conclusions

weighted_conclusions = weighted_inference(facts, weighted_rules)
weighted_conclusions


{'nadcisnienie': 0.9, 'cukrzyca': 0.85}

g) Definicja zbior?w rozmytych (fuzzy sets)

In [9]:
def trimf(x, a, b, c):
    x = np.asarray(x)
    y = np.zeros_like(x, dtype=float)
    y = np.where((x <= a) | (x >= c), 0.0, y)
    y = np.where((x > a) & (x < b), (x - a) / (b - a), y)
    y = np.where((x == b), 1.0, y)
    y = np.where((x > b) & (x < c), (c - x) / (c - b), y)
    return y

def trapmf(x, a, b, c, d):
    x = np.asarray(x)
    y = np.zeros_like(x, dtype=float)
    y = np.where((x <= a) | (x >= d), 0.0, y)
    y = np.where((x >= b) & (x <= c), 1.0, y)
    y = np.where((x > a) & (x < b), (x - a) / (b - a), y)
    y = np.where((x > c) & (x < d), (d - x) / (d - c), y)
    return y

fuzzy_sets = {
    'age': {
        'young': lambda x: trapmf(x, 0, 0, 30, 40),
        'middle': lambda x: trimf(x, 30, 45, 60),
        'old': lambda x: trapmf(x, 55, 65, 120, 120)
    },
    'bmi': {
        'normal': lambda x: trapmf(x, 0, 0, 22, 25),
        'over': lambda x: trimf(x, 22, 27, 32),
        'obese': lambda x: trapmf(x, 30, 35, 60, 60)
    },
    'sbp': {
        'normal': lambda x: trapmf(x, 0, 0, 120, 130),
        'high': lambda x: trapmf(x, 130, 140, 200, 200)
    },
    'dbp': {
        'normal': lambda x: trapmf(x, 0, 0, 80, 85),
        'high': lambda x: trapmf(x, 85, 90, 130, 130)
    },
    'glucose': {
        'normal': lambda x: trapmf(x, 0, 0, 100, 110),
        'high': lambda x: trapmf(x, 110, 126, 300, 300)
    }
}
fuzzy_sets


{'age': {'young': <function __main__.<lambda>(x)>,
  'middle': <function __main__.<lambda>(x)>,
  'old': <function __main__.<lambda>(x)>},
 'bmi': {'normal': <function __main__.<lambda>(x)>,
  'over': <function __main__.<lambda>(x)>,
  'obese': <function __main__.<lambda>(x)>},
 'sbp': {'normal': <function __main__.<lambda>(x)>,
  'high': <function __main__.<lambda>(x)>},
 'dbp': {'normal': <function __main__.<lambda>(x)>,
  'high': <function __main__.<lambda>(x)>},
 'glucose': {'normal': <function __main__.<lambda>(x)>,
  'high': <function __main__.<lambda>(x)>}}

h) Regu?y rozmyte (fuzzy rules)

In [10]:
fuzzy_rules = [
    {
        'name': 'r1',
        'if': [('age', 'old'), ('bmi', 'obese')],
        'then': 'high'
    },
    {
        'name': 'r2',
        'if': [('sbp', 'high'), ('dbp', 'high')],
        'then': 'high'
    },
    {
        'name': 'r3',
        'if': [('glucose', 'high')],
        'then': 'high'
    },
    {
        'name': 'r4',
        'if': [('age', 'middle'), ('bmi', 'over')],
        'then': 'medium'
    },
    {
        'name': 'r5',
        'if': [('age', 'young'), ('bmi', 'normal')],
        'then': 'low'
    }
]
fuzzy_rules


[{'name': 'r1', 'if': [('age', 'old'), ('bmi', 'obese')], 'then': 'high'},
 {'name': 'r2', 'if': [('sbp', 'high'), ('dbp', 'high')], 'then': 'high'},
 {'name': 'r3', 'if': [('glucose', 'high')], 'then': 'high'},
 {'name': 'r4', 'if': [('age', 'middle'), ('bmi', 'over')], 'then': 'medium'},
 {'name': 'r5', 'if': [('age', 'young'), ('bmi', 'normal')], 'then': 'low'}]

i) Wnioskowanie rozmyte (Mamdani)

In [11]:
risk_universe = np.linspace(0, 100, 501)
risk_sets = {
    'low': lambda x: trapmf(x, 0, 0, 20, 40),
    'medium': lambda x: trimf(x, 30, 50, 70),
    'high': lambda x: trapmf(x, 60, 75, 100, 100)
}

def fuzzify(facts):
    mu = {}
    for var, sets in fuzzy_sets.items():
        mu[var] = {}
        for label, fn in sets.items():
            mu[var][label] = float(fn(np.array([facts[var]]))[0])
    return mu

def eval_rule(rule, mu):
    strengths = [mu[var][label] for var, label in rule['if']]
    return float(np.min(strengths)) if strengths else 0.0

def mamdani_inference(facts):
    mu = fuzzify(facts)
    aggregated = np.zeros_like(risk_universe, dtype=float)
    rule_strengths = []
    for r in fuzzy_rules:
        strength = eval_rule(r, mu)
        rule_strengths.append((r['name'], r['then'], strength))
        if strength > 0:
            consequent = risk_sets[r['then']](risk_universe)
            aggregated = np.maximum(aggregated, np.minimum(strength, consequent))
    return aggregated, rule_strengths, mu

agg, rule_strengths, mu = mamdani_inference(facts)
agg[:10]


C:\Users\mindgoner\AppData\Local\Temp\ipykernel_33452\1432597896.py:15: RuntimeWarning: divide by zero encountered in divide
  y = np.where((x > a) & (x < b), (x - a) / (b - a), y)
C:\Users\mindgoner\AppData\Local\Temp\ipykernel_33452\1432597896.py:16: RuntimeWarning: divide by zero encountered in divide
  y = np.where((x > c) & (x < d), (d - x) / (d - c), y)
C:\Users\mindgoner\AppData\Local\Temp\ipykernel_33452\1432597896.py:16: RuntimeWarning: invalid value encountered in divide
  y = np.where((x > c) & (x < d), (d - x) / (d - c), y)
C:\Users\mindgoner\AppData\Local\Temp\ipykernel_33452\1432597896.py:15: RuntimeWarning: invalid value encountered in divide
  y = np.where((x > a) & (x < b), (x - a) / (b - a), y)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

j) Defuzyfikacja

In [12]:
def defuzzify_centroid(universe, mf):
    num = np.trapz(universe * mf, universe)
    den = np.trapz(mf, universe)
    return float(num / den) if den != 0 else 0.0

risk_score = defuzzify_centroid(risk_universe, agg)
risk_score


50.86666666666667

k) Wyja?nienie lokalne decyzji

In [13]:
local_explanation = pd.DataFrame(
    [(name, concl, strength) for name, concl, strength in rule_strengths if strength > 0],
    columns=['rule', 'risk', 'activation']
)
local_explanation


,rule,risk,activation
0,r3,high,1.0
1,r5,low,1.0


l) Wyja?nienie globalne systemu

In [14]:
def global_explanation(df):
    counts = {r['name']: 0 for r in fuzzy_rules}
    influence = {r['name']: 0.0 for r in fuzzy_rules}
    for _, row in df.iterrows():
        facts = facts_from_row(row, col_map)
        _, strengths, _ = mamdani_inference(facts)
        for name, _, strength in strengths:
            if strength > 0:
                counts[name] += 1
                influence[name] += strength
    res = pd.DataFrame({
        'rule': list(counts.keys()),
        'activation_count': list(counts.values()),
        'total_influence': [influence[k] for k in counts.keys()]
    })
    res = res.sort_values(['total_influence','activation_count'], ascending=False).reset_index(drop=True)
    return res

global_expl = global_explanation(df)
global_expl


C:\Users\mindgoner\AppData\Local\Temp\ipykernel_33452\1432597896.py:15: RuntimeWarning: divide by zero encountered in divide
  y = np.where((x > a) & (x < b), (x - a) / (b - a), y)
C:\Users\mindgoner\AppData\Local\Temp\ipykernel_33452\1432597896.py:16: RuntimeWarning: divide by zero encountered in divide
  y = np.where((x > c) & (x < d), (d - x) / (d - c), y)
C:\Users\mindgoner\AppData\Local\Temp\ipykernel_33452\1432597896.py:16: RuntimeWarning: invalid value encountered in divide
  y = np.where((x > c) & (x < d), (d - x) / (d - c), y)
C:\Users\mindgoner\AppData\Local\Temp\ipykernel_33452\1432597896.py:15: RuntimeWarning: invalid value encountered in divide
  y = np.where((x > a) & (x < b), (x - a) / (b - a), y)


,rule,activation_count,total_influence
0,r3,20,18.437500
1,r2,11,9.100000
2,r4,11,4.853333
3,r5,5,3.600000
4,r1,2,1.420000
